## 리뷰 감정분석
* 영화별 상위 500개 리뷰데이터를 테스트 데이터(예측 데이터)로 사용
* 나머지 리뷰 데이터를 감정분석 모델링하는데 사용

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_review=pd.read_csv("review_final_final.csv", encoding='cp949')
#review에 null값 제거
df_review=df_review[df_review['review'].notnull()]
df_review.head()

,id,rate,review,title
0,화늠(nhh5****),5,댓글 알바가 판을 치는군요 보면 압니다..,싱크홀
1,님(3328****),2,휴~ 90년대 개그와 cg…긴장감 전혀 없는 재난영화??,싱크홀
2,오이건축(jeon****),2,"두서도 없고,맥락도 없고,감동도 없고,유머만 조금,우주전쟁영화도 개연성이 있는데, ...",싱크홀
3,비즈(inab****),5,기대보다 별로였어요..,싱크홀
4,xovk****,8,승원이형 오랜만에 무도 한 편 더 찍으셨네요^^,싱크홀


In [3]:
# Group 짓기(moviename를 기준으로 묶기)
grouped = df_review.groupby("title")

In [4]:
df_review.groupby(['title'])['rate'].count().sort_values()

title
앵커                 962
디어 에반 핸슨          1004
스텔라               1047
귀문                1088
하우스 오브 구찌         1129
                 ...  
덩케르크             17549
스파이더맨: 노 웨이 홈    18809
범죄도시2            25056
너의 이름은.          31919
탑건: 매버릭          34067
Name: rate, Length: 80, dtype: int64

In [5]:
#영화별로 상위 리뷰 500개씩만 추출 -> 감정분석 예측데이터로 사용할 예정
df_500reviews_per_movie=None

for i,j in grouped:
    df_m=df_review[df_review['title']==i]
    df_m=df_m.iloc[:500,:]
    df_500reviews_per_movie=pd.concat([df_500reviews_per_movie,df_m])

In [6]:
df_500reviews_per_movie.shape

(40000, 4)

In [7]:
#영화별로 500개씩 잘 추출이 되었는지 확인
df_500reviews_per_movie.groupby(['title'])['review'].count()

title
007 노 타임 투 다이    500
가을의 전설           500
강릉               500
경관의 피            500
고스트버스터즈 라이즈      500
                ... 
해피 뉴 이어          500
헤어질 결심           500
호빗 : 뜻밖의 여정      500
호빗 : 스마우그의 폐허    500
호빗: 다섯 군대 전투     500
Name: review, Length: 80, dtype: int64

In [8]:
df_500reviews_per_movie.to_csv('감정분석 테스트 데이터.csv',index=False)
df_500reviews_per_movie.head()

,id,rate,review,title
75168,Fantasy(tbxo****),9,"굿바이, 다니엘 크레이그",007 노 타임 투 다이
75169,Debussy(kang****),1,처음 10분은 정말 007 역대급 작품이 나오는 줄 알았다.. 근데 그게 끝이였다..,007 노 타임 투 다이
75170,milk****,7,아니 감독양반 아무리 그래도 개간지 007 형님에게 도게자가 웬말이여..,007 노 타임 투 다이
75171,묵(vlue****),2,제임스본드를 그런식으로 퇴장시켜야만 했을까?별볼일없는 등장인물들에게 촬영분량을 주어...,007 노 타임 투 다이
75172,로저페더러(alka****),1,크레이그 007 중 최악의 작품. 후쿠나가가 007을 쓰레기로 만들었군요. 이 작품...,007 노 타임 투 다이


In [9]:
ind=df_500reviews_per_movie.index
ind=ind.values.tolist()

In [10]:
#학습 데이터 생성(전체 데이터에서 학습데이터를 제거한 나머지 리뷰 데이터)
df_train_review=df_review.drop(ind)
df_train_review.head()

,id,rate,review,title
503,zerodefect(lsh5****),1,7광구감독.. 말 다햇음,싱크홀
504,lbj8****,1,아... 영화학과 4학년생이 써도 이 시나리오 보다는 잘 쓸듯...아... 배우들의...,싱크홀
505,임성락(rakg****),2,처음에는 코미디 영화인줄알았다가 조금있으니 재난영화였다가 조금있으니 슬픈영화였다가 ...,싱크홀
506,카스파로(casp****),10,웃을일 하나 없는 이시국에 많이 웃고 나왔습니다. 영화는 이런 맛에 봐야 하지 싶은...,싱크홀
507,곽소영(cs20****),10,진짜 재미있는 포인트들이 많고 스토리도 좋고 너무 재미있었습니다 ㅠ 저는 영화 ‘ㅇ...,싱크홀


In [12]:
print('전체 데이터 크기 :',df_review.shape)
print('테스트 데이터 크기 :',df_500reviews_per_movie.shape)
print('학습 데이터 크기 :',df_train_review.shape)

전체 데이터 크기 : (478623, 4)
테스트 데이터 크기 : (40000, 4)
학습 데이터 크기 : (438623, 4)


In [13]:
df_train_review.to_csv('감정분석 학습데이터.csv',index=False)

In [14]:
#감정분석
# 사용할 library를 먼저 모두 import
import pandas as pd
import numpy as np
import konlpy
from sklearn.feature_extraction.text import CountVectorizer   # tf-idf 방식을 사용하려면 대신 TfidfVectorizer를 import
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re

In [15]:
# 텍스트를 tokenize해서 adjective, verb, noun만 추출하는 함수

def tokenize_korean_text(text): 
  text_filtered = re.sub('[^,.?!\w\s]','', text)

  okt = konlpy.tag.Okt() 
  Okt_morphs = okt.pos(text_filtered) 

  words = []
  for word, pos in Okt_morphs:
    if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
      words.append(word)

  words_str = ' '.join(words)
  return words_str

In [16]:
X_texts = []
y = []

for star, comment in zip(df_train_review['rate'],df_train_review['review']):
  if 4 <= star <= 7: 
    continue  
     # 평점이 4~7인 영화는 애매하기 때문에 학습데이터로 사용하지 않음

  tokenized_comment = tokenize_korean_text(comment)  # 위에서 만들었던 함수로 comment 쪼개기
  X_texts.append(tokenized_comment)

  y.append(1 if star > 7 else -1)
    # 평점이 8 이상이면(8,9,10) 값을 1로 지정 (positive)
    # 평점이 3 이하이면(1,2,3) 값을 -1로 지정 (negative)

In [17]:
# train_test_split
X_train_texts, X_test_texts, y_train, y_test = train_test_split(X_texts, y, test_size=0.2, random_state=0)

In [18]:
# CountVectorizer로 vector화
tf_vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1))
X_train_tf = tf_vectorizer.fit_transform(X_train_texts)  # training data에 맞게 fit & training data를 transform
X_test_tf = tf_vectorizer.transform(X_test_texts) # test data를 transform

vocablist = [word for word, number in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]  # 단어들을 번호 기준 내림차순으로 저장

In [19]:
#학습
model = LogisticRegression(C=0.1, penalty='l2', random_state=0)
model.fit(X_train_tf, y_train)

c:\users\samsung\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1, random_state=0)

In [20]:
#예측
y_test_pred = model.predict(X_test_tf)

print('Misclassified samples: {} out of {}'.format((y_test_pred != y_test).sum(), len(y_test)))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_test_pred)))  # model.score(X_test_tf, y_test)로 계산해도 됨

Misclassified samples: 6507 out of 73999
Accuracy: 0.91


In [21]:
#예측에 중요한 역할을 하는 단어들 확인
coefficients = model.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:10]:
  print('{0:}({1:.3f})'.format(vocablist[word_num], coef))

print('\n부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-10:]: 
  print('{0:}({1:.3f})'.format(vocablist[word_num], coef))

긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)
재밌었어요(2.621)
재밌었음(2.517)
재밌어요(2.425)
재밌는데(2.237)
존잼(2.160)
재미있었어요(2.075)
최고(2.051)
꿀잼(2.008)
좋았어요(1.963)
시키지(1.948)

부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)
쓰레기(-2.027)
잤다(-2.033)
불면증(-2.061)
졸작(-2.070)
아깝다(-2.218)
아까(-2.466)
재미없어요(-2.735)
노잼(-2.927)
재미없음(-3.007)
최악(-3.809)


In [22]:
# 긍정/부정 테스트용 함수 생성
list_result=[]
def guess_good_or_bad(text):
    text_filtered = text.replace('.', '').replace(',','').replace("'","").replace('·', ' ').replace('=','') 
    okt = konlpy.tag.Okt() 
    Okt_morphs = okt.pos(text_filtered) 
    words = []
    for word, pos in Okt_morphs:
        if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
            words.append(word)
            
    words_str = ' '.join(words)
    new_text_tf = tf_vectorizer.transform([words_str])
    
    if model.predict(new_text_tf) == 1:
        list_result.append('긍정')
    else:
        list_result.append('부정')
    
    return list_result

In [23]:
df_500reviews_per_movie.head()

,id,rate,review,title
75168,Fantasy(tbxo****),9,"굿바이, 다니엘 크레이그",007 노 타임 투 다이
75169,Debussy(kang****),1,처음 10분은 정말 007 역대급 작품이 나오는 줄 알았다.. 근데 그게 끝이였다..,007 노 타임 투 다이
75170,milk****,7,아니 감독양반 아무리 그래도 개간지 007 형님에게 도게자가 웬말이여..,007 노 타임 투 다이
75171,묵(vlue****),2,제임스본드를 그런식으로 퇴장시켜야만 했을까?별볼일없는 등장인물들에게 촬영분량을 주어...,007 노 타임 투 다이
75172,로저페더러(alka****),1,크레이그 007 중 최악의 작품. 후쿠나가가 007을 쓰레기로 만들었군요. 이 작품...,007 노 타임 투 다이


In [24]:
#학습데이터를 제외한 나머지 데이터에 대해서 감성분석 진행
for i in range(len(df_500reviews_per_movie)):
    str_review=str(df_500reviews_per_movie.iloc[i,2])
    guess_good_or_bad(str_review)

In [25]:
list_result

['긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '부정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '부정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '부정',
 '부정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',
 '긍정',

In [26]:
df_500reviews_per_movie['감성분석 결과']=list_result
df_500reviews_per_movie.head(30)

,id,rate,review,title,감성분석 결과
75168,Fantasy(tbxo****),9,"굿바이, 다니엘 크레이그",007 노 타임 투 다이,긍정
75169,Debussy(kang****),1,처음 10분은 정말 007 역대급 작품이 나오는 줄 알았다.. 근데 그게 끝이였다..,007 노 타임 투 다이,긍정
75170,milk****,7,아니 감독양반 아무리 그래도 개간지 007 형님에게 도게자가 웬말이여..,007 노 타임 투 다이,긍정
75171,묵(vlue****),2,제임스본드를 그런식으로 퇴장시켜야만 했을까?별볼일없는 등장인물들에게 촬영분량을 주어...,007 노 타임 투 다이,부정
75172,로저페더러(alka****),1,크레이그 007 중 최악의 작품. 후쿠나가가 007을 쓰레기로 만들었군요. 이 작품...,007 노 타임 투 다이,부정
75173,a bittersweet life(jinc****),9,난 아직도 악당들의 동기가 무엇인지 잘 모르겠다. 가족들은 왜 납치했으며 왜 화학전...,007 노 타임 투 다이,부정
75174,escuro(id44****),7,일본애들은 꼭 남의 음식에 와사비를 뿌린단 말이에요,007 노 타임 투 다이,긍정
75175,환웅천황(vita****),9,상영되고 노래나오기까지 20여분 만 봐도 아깝지 않을 정도로 강렬한 액션이 압도합니...,007 노 타임 투 다이,긍정
75176,dnemdnem(bigf****),8,크레이그의 본드를 마무리 한다는 점에서 의리로는 볼 만하지만 이전 007과는 다른 ...,007 노 타임 투 다이,긍정
75177,sd68****,2,"알 수 없는 전개, 석연치 않은 빌런, 천애고아의 가족애 007 / 넷플릭스 영국드...",007 노 타임 투 다이,부정


In [27]:
df_500reviews_per_movie.to_csv('학습데이터(영화별 상위500개씩) 감정분석 결과.csv',index=False)

In [28]:
df_500reviews_per_movie.head(10)

,id,rate,review,title,감성분석 결과
75168,Fantasy(tbxo****),9,"굿바이, 다니엘 크레이그",007 노 타임 투 다이,긍정
75169,Debussy(kang****),1,처음 10분은 정말 007 역대급 작품이 나오는 줄 알았다.. 근데 그게 끝이였다..,007 노 타임 투 다이,긍정
75170,milk****,7,아니 감독양반 아무리 그래도 개간지 007 형님에게 도게자가 웬말이여..,007 노 타임 투 다이,긍정
75171,묵(vlue****),2,제임스본드를 그런식으로 퇴장시켜야만 했을까?별볼일없는 등장인물들에게 촬영분량을 주어...,007 노 타임 투 다이,부정
75172,로저페더러(alka****),1,크레이그 007 중 최악의 작품. 후쿠나가가 007을 쓰레기로 만들었군요. 이 작품...,007 노 타임 투 다이,부정
75173,a bittersweet life(jinc****),9,난 아직도 악당들의 동기가 무엇인지 잘 모르겠다. 가족들은 왜 납치했으며 왜 화학전...,007 노 타임 투 다이,부정
75174,escuro(id44****),7,일본애들은 꼭 남의 음식에 와사비를 뿌린단 말이에요,007 노 타임 투 다이,긍정
75175,환웅천황(vita****),9,상영되고 노래나오기까지 20여분 만 봐도 아깝지 않을 정도로 강렬한 액션이 압도합니...,007 노 타임 투 다이,긍정
75176,dnemdnem(bigf****),8,크레이그의 본드를 마무리 한다는 점에서 의리로는 볼 만하지만 이전 007과는 다른 ...,007 노 타임 투 다이,긍정
75177,sd68****,2,"알 수 없는 전개, 석연치 않은 빌런, 천애고아의 가족애 007 / 넷플릭스 영국드...",007 노 타임 투 다이,부정


In [29]:
df_500reviews_per_movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 75168 to 193294
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       40000 non-null  object
 1   rate     40000 non-null  int64 
 2   review   40000 non-null  object
 3   title    40000 non-null  object
 4   감성분석 결과  40000 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


In [30]:
#감정분석 결과 긍정적인데 실제 평점은 3점 이하인 댓글들(리뷰들)
df_pos=df_500reviews_per_movie[(df_500reviews_per_movie['감성분석 결과']=='긍정')&(df_500reviews_per_movie['rate']<=3)]
df_pos.to_csv('긍정인데 평점 낮은 리뷰.csv',index=False)

In [31]:
#감정분석 결과 부정적인데 실제 평점은 8점 이상인 댓글들(리뷰들)
df_neg=df_500reviews_per_movie[(df_500reviews_per_movie['감성분석 결과']=='부정')&(df_500reviews_per_movie['rate']>=8)]
df_neg.to_csv('부정인데 평점 높은 리뷰.csv',index=False)

In [32]:
df_pos

,id,rate,review,title,감성분석 결과
75169,Debussy(kang****),1,처음 10분은 정말 007 역대급 작품이 나오는 줄 알았다.. 근데 그게 끝이였다..,007 노 타임 투 다이,긍정
75181,gang****,1,도게자 욱일기 감독아 국뽕영화찍냐,007 노 타임 투 다이,긍정
75182,파슬리(hars****),2,일본 감독이 다 망쳐놨음 ㅜㅜ,007 노 타임 투 다이,긍정
75184,소현(sohy****),2,그냥 스카이폴에서 끝났어야했다.,007 노 타임 투 다이,긍정
75185,ㅇ(hiye****),2,속상하다 크레이그 본드의 끝이 이래서...캐붕의 끝을 달린 영화.,007 노 타임 투 다이,긍정
...,...,...,...,...,...
192929,치아(onen****),1,"1편,2편 재밌게 봤다. 그 2편도 재밌게 봤었다. 3편. 보는중에 욕이 나왔다. ...",호빗: 다섯 군대 전투,긍정
192931,tndi****,1,예고편서 나오는 엘프활부대 활쏘는장면 어따 팔아먹었음?,호빗: 다섯 군대 전투,긍정
192961,메아(areo****),1,평점 조절용. 반지의 제왕 모든 시리즈 중 가장 못만든 영화.,호빗: 다섯 군대 전투,긍정
193090,강맹(luck****),1,맨칼쌈짓거리 돈이 아깝네 이런건집에서 봐야하는 영화 고만 울궈라이늠들아,호빗: 다섯 군대 전투,긍정


In [33]:
df_neg

,id,rate,review,title,감성분석 결과
75173,a bittersweet life(jinc****),9,난 아직도 악당들의 동기가 무엇인지 잘 모르겠다. 가족들은 왜 납치했으며 왜 화학전...,007 노 타임 투 다이,부정
75197,송지수(ebhl****),8,노래나오고 30분정도까진 정말 007다운 멋진 마무리가 되겠구나 했는데.. 마지막 ...,007 노 타임 투 다이,부정
75339,하루키(mkha****),8,"007을 이런 식으로 보낸다는 게 참...내용도 그닥 와닿지도 않고, 부성애는 공감...",007 노 타임 투 다이,부정
75380,햇살처럼(guru****),8,음 다니엘의 007 마지막편이었는데 일본 감독?이라서 그런지 왜색도 짖고 조금 실망...,007 노 타임 투 다이,부정
75476,치즈(chzt****),8,기대한 것 보다는 별로였어요,007 노 타임 투 다이,부정
...,...,...,...,...,...
193084,wndo****,8,기대한만큼은 그닥 아닌듯 ㅎ,호빗: 다섯 군대 전투,부정
193144,대한건아(dhdl****),10,후... 확장판봤는데 이거 너무 심하게 편집된거 아닌지 모르겠네요/// 너무하네 진...,호빗: 다섯 군대 전투,부정
193162,이라이자(funk****),10,"중간부터 한참을 대사없는 전쟁신이었는데도 지루함을 못 느꼈다. 생일날 보았는데, 정...",호빗: 다섯 군대 전투,부정
193206,jaha****,10,급전개랑 전략적인 요소가 없는 개판 싸움만 빼면ㅋㅋㅋㅋ 볼만합니다 CG굿!,호빗: 다섯 군대 전투,부정


## 가짜 리뷰 제거 

In [34]:
pos_index=df_pos.index.tolist()
neg_index=df_neg.index.tolist()

In [35]:
df_review.to_csv('review_final_nonnull.csv',index=True)

In [36]:
#이상리뷰 인덱스
index_review=pos_index+neg_index
index_review

[75169,
 75181,
 75182,
 75184,
 75185,
 75187,
 75194,
 75200,
 75212,
 75251,
 75265,
 75277,
 75290,
 75299,
 75313,
 75323,
 75331,
 75347,
 75349,
 75367,
 75402,
 75423,
 75438,
 75440,
 75450,
 75456,
 75538,
 75547,
 75557,
 75559,
 75561,
 75590,
 75594,
 75597,
 75599,
 75629,
 75659,
 75661,
 75668,
 104556,
 104557,
 104582,
 104596,
 104597,
 104602,
 104607,
 104628,
 104645,
 104653,
 104657,
 104675,
 104694,
 104754,
 104766,
 104767,
 104768,
 104784,
 104821,
 104872,
 104874,
 104878,
 104880,
 104881,
 104883,
 104884,
 136872,
 136878,
 136879,
 136880,
 136881,
 136883,
 136884,
 136886,
 136887,
 136894,
 136914,
 136923,
 136926,
 136945,
 136957,
 136983,
 137005,
 137031,
 137043,
 137047,
 137050,
 137057,
 137058,
 137059,
 137102,
 137104,
 137108,
 137109,
 137123,
 137127,
 137132,
 137134,
 137136,
 137137,
 137141,
 137145,
 137149,
 137155,
 137156,
 137159,
 137161,
 137162,
 137212,
 137222,
 137225,
 137230,
 137234,
 137236,
 137239,
 137241,
 137

In [37]:
#학습 데이터 인덱스도 리뷰 보정할 때 빼야함
df_train_review.head()

,id,rate,review,title
503,zerodefect(lsh5****),1,7광구감독.. 말 다햇음,싱크홀
504,lbj8****,1,아... 영화학과 4학년생이 써도 이 시나리오 보다는 잘 쓸듯...아... 배우들의...,싱크홀
505,임성락(rakg****),2,처음에는 코미디 영화인줄알았다가 조금있으니 재난영화였다가 조금있으니 슬픈영화였다가 ...,싱크홀
506,카스파로(casp****),10,웃을일 하나 없는 이시국에 많이 웃고 나왔습니다. 영화는 이런 맛에 봐야 하지 싶은...,싱크홀
507,곽소영(cs20****),10,진짜 재미있는 포인트들이 많고 스토리도 좋고 너무 재미있었습니다 ㅠ 저는 영화 ‘ㅇ...,싱크홀


In [38]:
list_df_train_review=df_train_review.index.tolist()

In [39]:
#Null값이 있는 인덱스도 제거해줘야 함
df_review_=pd.read_csv("review_final_final.csv", encoding='cp949')
df_review_null=df_review_[df_review_['review'].isnull()]

In [40]:
df_review_null.head()

,id,rate,review,title
225,Deby(78_h****),4,NaN,싱크홀
253,6uoas(seon****),10,NaN,싱크홀
277,alsw****,10,NaN,싱크홀
510,스파이(wldu****),4,NaN,싱크홀
517,Yes babyyyyy(jung****),1,NaN,싱크홀


In [41]:
null_list=df_review_null.index.tolist()

In [42]:
null_list

[225,
 253,
 277,
 510,
 517,
 551,
 555,
 559,
 658,
 728,
 746,
 817,
 889,
 897,
 900,
 998,
 1015,
 1123,
 1137,
 1139,
 1224,
 1389,
 1435,
 1461,
 1492,
 1496,
 1530,
 1553,
 1576,
 1626,
 1650,
 1675,
 1676,
 1694,
 1696,
 1700,
 1747,
 1759,
 1760,
 1763,
 1775,
 1879,
 1910,
 1928,
 1984,
 2012,
 2074,
 2171,
 2180,
 2189,
 2197,
 2201,
 2357,
 2411,
 2455,
 2500,
 2543,
 2553,
 2588,
 2629,
 2632,
 2642,
 2656,
 2668,
 2694,
 2712,
 2726,
 2730,
 2734,
 2743,
 2745,
 2750,
 2789,
 2793,
 2811,
 2824,
 2826,
 2832,
 2848,
 2863,
 2886,
 2904,
 2924,
 2978,
 2994,
 3002,
 3038,
 3045,
 3085,
 3088,
 3202,
 3217,
 3231,
 3241,
 3294,
 3318,
 3327,
 3335,
 3343,
 3350,
 3360,
 3375,
 3477,
 3527,
 3547,
 3548,
 3565,
 3632,
 3645,
 3646,
 3649,
 3669,
 3679,
 3692,
 3755,
 3785,
 3791,
 3792,
 3847,
 3862,
 3865,
 3972,
 3974,
 3977,
 4011,
 4027,
 4036,
 4227,
 4294,
 4349,
 4384,
 4413,
 4546,
 4551,
 4577,
 4626,
 4673,
 4683,
 4802,
 4805,
 4808,
 4821,
 4843,
 4863,
 4866,
 

In [43]:
list_drop_index= pos_index + neg_index + list_df_train_review
list_drop_index

[75169,
 75181,
 75182,
 75184,
 75185,
 75187,
 75194,
 75200,
 75212,
 75251,
 75265,
 75277,
 75290,
 75299,
 75313,
 75323,
 75331,
 75347,
 75349,
 75367,
 75402,
 75423,
 75438,
 75440,
 75450,
 75456,
 75538,
 75547,
 75557,
 75559,
 75561,
 75590,
 75594,
 75597,
 75599,
 75629,
 75659,
 75661,
 75668,
 104556,
 104557,
 104582,
 104596,
 104597,
 104602,
 104607,
 104628,
 104645,
 104653,
 104657,
 104675,
 104694,
 104754,
 104766,
 104767,
 104768,
 104784,
 104821,
 104872,
 104874,
 104878,
 104880,
 104881,
 104883,
 104884,
 136872,
 136878,
 136879,
 136880,
 136881,
 136883,
 136884,
 136886,
 136887,
 136894,
 136914,
 136923,
 136926,
 136945,
 136957,
 136983,
 137005,
 137031,
 137043,
 137047,
 137050,
 137057,
 137058,
 137059,
 137102,
 137104,
 137108,
 137109,
 137123,
 137127,
 137132,
 137134,
 137136,
 137137,
 137141,
 137145,
 137149,
 137155,
 137156,
 137159,
 137161,
 137162,
 137212,
 137222,
 137225,
 137230,
 137234,
 137236,
 137239,
 137241,
 137

In [44]:
len(list_drop_index)

442136

In [45]:
df_drop_index=pd.DataFrame({'drop_index':list_drop_index})
df_drop_index.head()

,drop_index
0,75169
1,75181
2,75182
3,75184
4,75185


In [46]:
df_drop_index.to_csv('case1_drop_index.csv',index=False)

In [47]:
df_drop_index_case3=pd.DataFrame({'drop_index':null_list})
df_drop_index_case3.head()

,drop_index
0,225
1,253
2,277
3,510
4,517


In [48]:
df_drop_index_case3.to_csv('case3_drop_index.csv',index=False)

In [49]:
dup_index=pd.read_csv('dup_drop_index.csv',encoding='cp949')
case1=dup_index['drop_index'].values.tolist()

In [50]:
case2=list_drop_index
case3=null_list

In [51]:
final_drop_index=case1+case2+case3

In [52]:
len(final_drop_index)

468637

In [53]:
df_review_.drop(final_drop_index,inplace=True)

In [54]:
df_review_.head()

,id,rate,review,title
0,화늠(nhh5****),5,댓글 알바가 판을 치는군요 보면 압니다..,싱크홀
1,님(3328****),2,휴~ 90년대 개그와 cg…긴장감 전혀 없는 재난영화??,싱크홀
2,오이건축(jeon****),2,"두서도 없고,맥락도 없고,감동도 없고,유머만 조금,우주전쟁영화도 개연성이 있는데, ...",싱크홀
3,비즈(inab****),5,기대보다 별로였어요..,싱크홀
4,xovk****,8,승원이형 오랜만에 무도 한 편 더 찍으셨네요^^,싱크홀


In [55]:
drop_id=pd.read_csv('case2_drop_name_list.csv',encoding='cp949')
drop_id_list=drop_id['name'].values.tolist()

In [56]:
df_final=None
for i in range(len(drop_id_list)):
    tf=df_review_[df_review_['id']==drop_id_list[i]]
    df_final=pd.concat([df_final,tf])

In [57]:
dup_id_index=df_final.index.tolist()

In [58]:
df_review_.drop(dup_id_index,inplace=True)

In [59]:
df_review_.to_csv('final.csv',index=False)

### 가짜리뷰 제거 후(리뷰 보정후) 영화별 리뷰 

In [60]:
df_review_.groupby(['title'])['rate'].mean()

title
007 노 타임 투 다이    6.334086
가을의 전설           8.698851
강릉               6.387179
경관의 피            7.211207
고스트버스터즈 라이즈      7.632258
                   ...   
해피 뉴 이어          6.182033
헤어질 결심           9.501035
호빗 : 뜻밖의 여정      9.344754
호빗 : 스마우그의 폐허    9.477223
호빗: 다섯 군대 전투     9.243534
Name: rate, Length: 80, dtype: float64

In [61]:
review_real=df_review.groupby(['title'])['rate'].mean().values.tolist()

In [62]:
df_review_final=df_review_.groupby(['title'])['rate'].mean().reset_index()

In [63]:
df_review_final['real_review']=review_real

In [64]:
df_review_final.rename(columns={'title':'title','rate':'corrected_rate','real_review':'real_rate'},inplace=True)

In [65]:
df_review_final.to_csv('rate_correction.csv',index=False)

In [66]:
df_review_final['corrected_rate']-df_review_final['real_rate']

0    -0.328475
1     0.030479
2     1.176215
3     0.045860
4     0.013650
        ...   
75   -0.759988
76    0.898162
77    0.885554
78    0.915799
79    0.703463
Length: 80, dtype: float64